# 1. Problema de Negócio

O conjunto de dados foi coletado por um 
período de 10 minutos por cerca de 5 meses. As condições de temperatura e 
umidade da casa foram monitoradas com uma rede de sensores sem fio ZigBee. 
Cada nó sem fio transmitia as condições de temperatura e umidade em torno 
de 3 min. Em seguida, a média dos dados foi calculada para períodos de 10 minutos. 

Os dados de energia foram registrados a cada 10 minutos com medidores de 
energia de barramento m. O tempo da estação meteorológica mais próxima do 
aeroporto (Aeroporto de Chievres, Bélgica) foi baixado de um conjunto de dados 
públicos do Reliable Prognosis (rp5.ru) e mesclado com os conjuntos de dados 
experimentais usando a coluna de data e hora. Duas variáveis aleatórias foram 
incluídas no conjunto de dados para testar os modelos de regressão e filtrar os 
atributos não preditivos (parâmetros).

| Feature     | Descrição                                          | Unidade        |
|-------------|----------------------------------------------------|----------------|
| date        | Data no formato ano-mês-dia hora:minutos:segundos. |                |
| Appliances  | Consumo de energia. Variavel Target.               | Wh (Watt-Hora) |
| lights      | Consumo de energia de luminárias.                  | Wh (Watt-Hora) |
| T1          | Temperatura na Cozinha.                            | Celsius        |
| RH1         | Umidade Relativa na Cozinha.                       | %              |
| T2          | Temperatura na Sala de Estar.                      | Celsius        |
| RH2         | Umidade Relativa na Sala de Estar.                 | %              |
| T3          | Temperatura na Lavanderia.                         | Celsius        |
| RH3         | Umidade Relativa na Lavanderia.                    | %              |
| T4          | Temperatura no Escritório.                         | Celsius        |
| RH4         | Umidade Relativa no Escritório.                    | %              |
| T5          | Temperatura no Banheiro.                           | Celsius        |
| RH5         | Umidade Relativa no Banheiro.                      | %              |
| T6          | Temperatura Externa Lado Norte.                    | Celsius        |
| RH6         | Umidade Relativa Externa Lado Norte.               | %              |
| T7          | Temperatura na Sala de Passar Roupa.               | Celsius        |
| RH7         | Umidade Relativa na Sala de Passar Roupa.          | %              |
| T8          | Temperatura no Quarto do Adolescente.              | Celsius        |
| RH8         | Umidade Relativa no Quarto do Adolescente.         | %              |
| T9          | Temperatura no Quarto dos Pais.                    | Celsius        |
| RH9         | Umidade Relativa no Quarto dos Pais.               | %              |
| T_out       | Temperatura Externa.                               | Celsius        |
| Press_mm_hg | Pressão.                                           | mm/hg          |
| RH_out      | Umidade Relativa Externa.                          | %              |
| Windspeed   | Velocidade do Vento.                               | m/s            |
| Visibility  | Visibilidade.                                      | km             |
| Tdewpoint   | Ponto de Saturação.                                | Celsius        |
| rv1         | Variável Randômica.                                |                |
| rv2         | Variável Randômica.                                |                |
| NSM         | Número sequencial de medição.                      |                |
| WeekStatus  | Indicativo de Dia da Semana ou Final de Semana.    |                |
| Day_of_week | Indicativo de Segunda à Domingo.                   |                |

# 2. Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import simplefilter

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Herikc Brecher" --iversions

Author: Herikc Brecher

matplotlib: 3.3.4
seaborn   : 0.11.1
pandas    : 1.2.4
numpy     : 1.19.5



## 2.1 Ambiente

In [8]:
simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
sns.set_theme()

In [9]:
seed_ = 194
np.random.seed(seed_)

# 3. Carregamento dos Dados

In [11]:
# Carregamento do dataset de treino e teste
dtTreino = pd.read_csv('data/training.csv')
dtTeste = pd.read_csv('data/testing.csv')

In [12]:
dtTreino.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM,WeekStatus,Day_of_week
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,61200,Weekday,Monday
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,61800,Weekday,Monday
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,62400,Weekday,Monday
3,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,63600,Weekday,Monday
4,2016-01-11 17:50:00,50,40,19.89,46.026667,19.2,44.500000,19.79,44.933333,18.890000,...,734.0,92.0,5.333333,43.833333,4.8,44.919484,44.919484,64200,Weekday,Monday


In [13]:
dtTeste.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM,WeekStatus,Day_of_week
0,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.79,45.000000,18.89,...,733.800000,92.000000,6.000000,51.5,5.000000,45.410389,45.410389,63000,Weekday,Monday
1,2016-01-11 18:00:00,60,50,19.890000,45.766667,19.200000,44.500000,19.79,44.900000,18.89,...,734.100000,92.000000,5.000000,40.0,4.700000,47.233763,47.233763,64800,Weekday,Monday
2,2016-01-11 18:40:00,230,70,19.926667,45.863333,19.356667,44.400000,19.79,44.900000,18.89,...,734.366667,91.333333,5.666667,40.0,4.633333,10.298729,10.298729,67200,Weekday,Monday
3,2016-01-11 18:50:00,580,60,20.066667,46.396667,19.426667,44.400000,19.79,44.826667,19.00,...,734.433333,91.166667,5.833333,40.0,4.616667,8.827838,8.827838,67800,Weekday,Monday
4,2016-01-11 19:30:00,100,10,20.566667,53.893333,20.033333,46.756667,20.10,48.466667,19.00,...,734.850000,89.500000,6.000000,40.0,4.350000,24.884962,24.884962,70200,Weekday,Monday


In [15]:
dtFull = pd.concat([dtTreino, dtTeste], axis = 0)

In [16]:
dtFull.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM,WeekStatus,Day_of_week
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,61200,Weekday,Monday
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,61800,Weekday,Monday
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,62400,Weekday,Monday
3,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,63600,Weekday,Monday
4,2016-01-11 17:50:00,50,40,19.89,46.026667,19.2,44.500000,19.79,44.933333,18.890000,...,734.0,92.0,5.333333,43.833333,4.8,44.919484,44.919484,64200,Weekday,Monday


In [17]:
print(dtTreino.shape, dtTeste.shape, dtFull.shape)

(14803, 32) (4932, 32) (19735, 32)


# 4. Analise Exploratoria

In [18]:
dtFull.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033,42907.129465
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634,24940.020831
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322,0.000000
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889,21600.000000
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653,43200.000000
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769,64200.000000
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530,85800.000000
